In [1]:
# Imports
import scipy.io
import numpy as np
import torch
import torch.nn as nn

In [2]:
# Load the .mat file
data = scipy.io.loadmat('data/condsForSimJ2moMuscles.mat')

# Extract condsForSim struct
conds_for_sim = data['condsForSim']

In [3]:
# Initialize lists to store data for all conditions
go_envelope_all = []
plan_all = []
muscle_all = []

# Get the number of conditions (rows) and delay durations (columns)
num_conditions, num_delays = conds_for_sim.shape

# Loop through each condition and extract data
for i in range(num_conditions):  # 27 conditions
    go_envelope_condition = []
    plan_condition = []
    muscle_condition = []

    for j in range(num_delays):  # 8 delay durations
        condition = conds_for_sim[i, j]

        go_envelope = condition['goEnvelope']
        plan = condition['plan']
        muscle = condition['muscle']

        go_envelope_condition.append(go_envelope)
        plan_condition.append(plan)
        muscle_condition.append(muscle)

    # Stack data for each condition
    go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))
    plan_all.append(torch.tensor(plan_condition, dtype=torch.float32))
    muscle_all.append(torch.tensor(muscle_condition, dtype=torch.float32))

# Stack data for all conditions
go_envelope_tensor = torch.stack(go_envelope_all)
plan_tensor = torch.stack(plan_all)
muscle_tensor = torch.stack(muscle_all)

# Reshape to merge the first two dimensions
go_envelope_tensor = go_envelope_tensor.reshape(-1, *go_envelope_tensor.shape[2:])
plan_tensor = plan_tensor.reshape(-1, *plan_tensor.shape[2:])
muscle_tensor = muscle_tensor.reshape(-1, *muscle_tensor.shape[2:])

# Print shapes
print(f"Go Envelope Tensor Shape: {go_envelope_tensor.shape}")
print(f"Plan Tensor Shape: {plan_tensor.shape}")
print(f"Muscle Tensor Shape: {muscle_tensor.shape}")

/tmp/ipykernel_204181/828212273.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))


Go Envelope Tensor Shape: torch.Size([216, 296, 1])
Plan Tensor Shape: torch.Size([216, 296, 15])
Muscle Tensor Shape: torch.Size([216, 296, 8])


In [4]:
# Define a RNN model

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out

# Assuming the sizes from data
input_size = 16 # Calculated based on input shapes
hidden_size = 64 
output_size = 8 # Based on the output shape

model = SimpleRNN(input_size, hidden_size, output_size)

In [5]:
# Adjust the shape of go_envelope_tensor
go_envelope_tensor_adjusted = go_envelope_tensor.squeeze(-1)  # Removes the last dimension

# Check dimensions after squeezing
if go_envelope_tensor_adjusted.dim() == plan_tensor.dim() - 1:
    # Add an extra dimension to go_envelope_tensor_adjusted to match plan_tensor
    go_envelope_tensor_adjusted = go_envelope_tensor_adjusted.unsqueeze(-1)

    # Now concatenate along the last dimension
    input_tensor = torch.cat((go_envelope_tensor_adjusted, plan_tensor), dim=-1)
else:
    raise RuntimeError("Dimension mismatch after adjustment")

In [6]:
# Split data into training and testing sets
train_size = int(0.8 * input_tensor.size(0))
train_input = input_tensor[:train_size]
test_input = input_tensor[train_size:]
train_target = muscle_tensor[:train_size]
test_target = muscle_tensor[train_size:]

# Verify the sizes
print(f"Train Input Size: {train_input.size()}")
print(f"Test Input Size: {test_input.size()}")
print(f"Train Target Size: {train_target.size()}")
print(f"Test Target Size: {test_target.size()}")

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train Input Size: torch.Size([172, 296, 16])
Test Input Size: torch.Size([44, 296, 16])
Train Target Size: torch.Size([172, 296, 8])
Test Target Size: torch.Size([44, 296, 8])


In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(train_input)
    loss = criterion(output, train_target)
    loss.backward()
    optimizer.step()

    # Evaluation on test data
    model.eval()
    with torch.no_grad():
        test_output = model(test_input)
        test_loss = criterion(test_output, test_target)

    # Print loss every few epochs for monitoring
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}, Test Loss: {test_loss.item()}')

# Save the model
torch.save(model.state_dict(), 'simple_model_checkpoint.pth')


Epoch [0/500], Loss: 0.06945484131574631, Test Loss: 0.06215358525514603
Epoch [10/500], Loss: 0.021324191242456436, Test Loss: 0.0196370966732502
Epoch [20/500], Loss: 0.019416367635130882, Test Loss: 0.01877027191221714
Epoch [30/500], Loss: 0.01791669800877571, Test Loss: 0.016670530661940575
Epoch [40/500], Loss: 0.017395811155438423, Test Loss: 0.01648167334496975
Epoch [50/500], Loss: 0.01716764271259308, Test Loss: 0.016113314777612686
Epoch [60/500], Loss: 0.016996093094348907, Test Loss: 0.016008485108613968
Epoch [70/500], Loss: 0.016899049282073975, Test Loss: 0.015784764662384987
Epoch [80/500], Loss: 0.016822611913084984, Test Loss: 0.01577206887304783
Epoch [90/500], Loss: 0.016755780205130577, Test Loss: 0.01563873328268528
Epoch [100/500], Loss: 0.016685090959072113, Test Loss: 0.01555646862834692
Epoch [110/500], Loss: 0.01658358983695507, Test Loss: 0.015401231124997139
Epoch [120/500], Loss: 0.016355616971850395, Test Loss: 0.015009786002337933
Epoch [130/500], Loss: